In [1]:
# con transformers de HuggingFace
from transformers import pipeline

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # te dirá si tienes GPU

In [2]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [6]:
texto = """
El Consejo Fiscal (CF) considera su deber reiterar la necesidad de iniciar un proceso de consolidación fiscal, que nos permita recuperar la fortaleza fiscal que la economía peruana tenía antes de la pandemia. Para ese objetivo, el CF hace un llamado a las autoridades del Legislativo y del Ejecutivo a actuar con extrema responsabilidad fiscal para no agravar la crisis fiscal producida por la COVID19. En particular, el CF recalca su preocupación ante las iniciativas legislativas formuladas por el Congreso de la República que afectarían de manera significativa la reconstrucción de las finanzas públicas a mediano plazo, a través del aumento del gasto público, la reducción de los ingresos fiscales y la mayor exposición a riesgos fiscales. De acuerdo con el análisis realizado por la Dirección de Estudios Macrofiscales de la Secretaría Técnica del CF (DEM-STCF), aproximadamente el 21 por ciento de los proyectos de ley presentados en el Congreso de la República en el periodo $2020^{1}$ afectarían negativamente a las finanzas públicas, de ser aprobados, incidiendo en un mayor déficit fiscal y, por ende, en una mayor deuda pública. El porcentaje de proyectos de ley con incidencia fiscal se incrementa a 31 por ciento si se considera únicamente a los proyectos de ley que ya cuentan con dictamen positivo. Las propuestas con mayor costo fiscal, y por ende con un significativo impacto negativo sobre las finanzas públicas, son las tres siguientes:
Finalmente, el CF recalca la importancia de mantener una senda sostenible de la deuda pública, que ha caracterizado a la política fiscal peruana en las últimas décadas, y que ha sido determinante para la estabilidad macroeconómica del país."""

etiquetas = ["advertencia crítica", "advertencia moderada", "neutral", "positiva"]

resultado = classifier(texto, candidate_labels=etiquetas)

print(resultado)

{'sequence': '\nEl Consejo Fiscal (CF) considera su deber reiterar la necesidad de iniciar un proceso de consolidación fiscal, que nos permita recuperar la fortaleza fiscal que la economía peruana tenía antes de la pandemia. Para ese objetivo, el CF hace un llamado a las autoridades del Legislativo y del Ejecutivo a actuar con extrema responsabilidad fiscal para no agravar la crisis fiscal producida por la COVID19. En particular, el CF recalca su preocupación ante las iniciativas legislativas formuladas por el Congreso de la República que afectarían de manera significativa la reconstrucción de las finanzas públicas a mediano plazo, a través del aumento del gasto público, la reducción de los ingresos fiscales y la mayor exposición a riesgos fiscales. De acuerdo con el análisis realizado por la Dirección de Estudios Macrofiscales de la Secretaría Técnica del CF (DEM-STCF), aproximadamente el 21 por ciento de los proyectos de ley presentados en el Congreso de la República en el periodo $2

In [10]:
import numpy as np

pesos = {
    "advertencia crítica": 0,
    "advertencia moderada": 1,
    "neutral": 1,
    "positiva": 1
}

# Extrae etiquetas y scores
labels = resultado['labels']
scores = resultado['scores']

# Calcula índice
indice = np.sum([pesos[label] * score for label, score in zip(labels, scores)])
print(f"Índice de tono fiscal: {indice:.2f}")


Índice de tono fiscal: 0.54


In [9]:
print(pesos)

{'advertencia crítica': -2, 'advertencia moderada': -1, 'neutral': 0, 'positiva': 1}
